## Behind the scenes: structural clustering of adjacency matrices
### Please note that the adjacency matrices themselves are computed in Method:Knowledge_graphs

In [2]:
import numpy as np
import networkx as nx
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from adjustText import adjust_text
from itertools import cycle

### Load relevant data

In [ ]:
######
depth = 3 # Options: -1 (unpruned), 1, 2, 3
with_tags = 'no' # Options: 'with' (yes) and 'no' (no)
######

filepath = f'../Method:Knowledge_graphs/data/triplets_no_cutoff/adj_matrices_reduced_rel_dim_pruned_{depth}_{with_tags}_tags.npz'
loaded_data = np.load(filepath, allow_pickle=False)
loaded_adj_matrices = {key: loaded_data[key] for key in loaded_data}
print(f"Finished loading adjacency matrices (depth={depth}, {with_tags} tags)") 
print(f"Shape: {loaded_adj_matrices['ARC'].shape}")

### Perform clustering and get Silhouette Scores

In [ ]:
# 1. Extract features
def extract_features_from_adj_matrices(adj_matrices):
    '''Flatten and concatenate the adjacency matrices'''
    flattened_matrices = [adj_matrix.flatten() for adj_matrix in adj_matrices]
    return np.concatenate(flattened_matrices)

firm_features = []
firm_names = []

for firm, adj_matrices in loaded_adj_matrices.items():
    if adj_matrices is not None:
        features = extract_features_from_adj_matrices(adj_matrices)
        firm_features.append(features)
        firm_names.append(firm)

firm_features = np.array(firm_features)

# 2. Remove features with no explanatory power (i.e. var = 0)
selector = VarianceThreshold(threshold=0.0)
firm_features_reduced = selector.fit_transform(firm_features)

print(f"Original number of features: {firm_features.shape[1]}")
print(f"Number of features after removing zero variance: {firm_features_reduced.shape[1]}")

# 3. Normalise features
scaler = StandardScaler()
firm_features_normalized = scaler.fit_transform(firm_features_reduced)
#np.save('data/firm_features.npy', firm_features_reduced)
np.save('data/firm_features_normalized.npy', firm_features_normalized)
np.save('data/firm_names.npy', firm_names)

# 4. Compute Silhouette scores
print("\nSilhouette Scores:")
firm_features_normalized = np.load('data/firm_features_normalized.npy')
max_K = 0
max_score = 0
for num_clusters in range(2,16):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(firm_features_normalized)
    silhouette_avg = silhouette_score(firm_features_normalized, kmeans_labels)
    if silhouette_avg > max_score:
        max_score = silhouette_avg
        max_K = num_clusters
    print(f"- K = {num_clusters}, Score = {silhouette_avg:.6f}")

print(f"\nRecommended number of clusters: {max_K} (Score: {max_score:.3f})")

### Visualise Clusters
##### The hyperparam n_clusters determines how many clusters (recommended K given above)

In [354]:
def make_kmeans_plot(n_clusters=2, depth=-1, with_tags='no', exclude_firms=None):
    if n_clusters < 2:
        raise KeyError("Please choose a K higher than 1")

    # Load data
    firm_features_normalized = np.load('data/firm_features_normalized.npy')
    firm_names = np.load('data/firm_names.npy')

    # Exclude specified firms
    if exclude_firms is not None:
        exclude_indices = [i for i, firm in enumerate(firm_names) if firm in exclude_firms]
        firm_features_normalized = np.delete(firm_features_normalized, exclude_indices, axis=0)
        firm_names = np.delete(firm_names, exclude_indices, axis=0)

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(firm_features_normalized)
    sil_score = silhouette_score(firm_features_normalized, kmeans_labels)

    # Apply PCA for dimensionality reduction
    pca = IncrementalPCA(n_components=2, batch_size=200)
    reduced_features = pca.fit_transform(firm_features_normalized)

    # Plot
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=kmeans_labels, cmap='viridis')

    texts = []
    for i, (firm, cluster) in enumerate(zip(firm_names, kmeans_labels)):
        texts.append(plt.text(reduced_features[i, 0], reduced_features[i, 1], firm, fontsize=10, alpha=0.75))

    adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.8))

    # Add info
    prune_text = 'no'
    if depth > 0:
        prune_text = depth
    textstr = f"{with_tags.capitalize()} tags\nPruned: {prune_text}\n\nNum clusters: {n_clusters}\nSil score: {sil_score:.3f}"
    props = dict(boxstyle='square,pad=0.5', facecolor='lightgrey', edgecolor='black', alpha=0.75)
    plt.text(0.04, 0.80, textstr, transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='bottom', horizontalalignment='left', bbox=props)

    # NOTE: the two first parameters in plt.text() determines vertical and horizontal position of grey box
    # Change these params to re-position grey box if it overlaps with data points
    # Recommended: 
    #   (0.04, 0.80) for top left
    #   (0.85, 0.80) for top right
    #   (0.85, 0.05) for bottom right

    filepath = f"results/kmeans_structural/kmeans_pruned_{depth}_{with_tags}_tags_{n_clusters}_clusters"
    if exclude_firms:
        filepath += f'_excluded{len(exclude_firms)}'
    filepath += f"_{str(int(round(sil_score, 2)*100))}"
    print(f"Saving plot to {filepath}")

    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.tight_layout()
    plt.savefig(filepath)
    plt.show()
    plt.close()

In [ ]:
exclude_firms = None
#exclude_firms = ['Ultra Safe Nuclear Corporation', 'Babcock and Wilcox', 'General Atomics']
#exclude_firms = ['Westinghouse', 'Framatome']
#exclude_firms = ['Westinghouse']
#exclude_firms = ['NuScale', 'Westinghouse']
#exclude_firms = ['Westinghouse', 'NuScale', 'GE Hitachi']

for K in [3]:
    make_kmeans_plot(n_clusters=K, depth=depth, with_tags=with_tags, exclude_firms=exclude_firms)

In [ ]:
# Apply Hierarchical KMeans clustering
firm_features_normalized = np.load('data/firm_features_normalized.npy')
firm_names = np.load('data/firm_names.npy')

# First level
kmeans_level_1 = KMeans(n_clusters=2, random_state=42)
first_level_labels = kmeans_level_1.fit_predict(firm_features_normalized)

second_level_labels = np.zeros_like(first_level_labels)

# Second-level clustering
for cluster in np.unique(first_level_labels):
    cluster_indices = np.where(first_level_labels == cluster)[0]

    if len(cluster_indices) < 3:
        print(f"Cluster {cluster} has too few points for further splitting.")
        continue

    best_score = -1
    best_n_clusters = 2

    print(f'Testing for Cluster {cluster}:')

    for n_clusters in range(2,7):
        try:
            kmeans_tmp = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            sub_cluster_labels = kmeans_tmp.fit_predict(firm_features_normalized[cluster_indices])
            score = silhouette_score(firm_features_normalized[cluster_indices], sub_cluster_labels)
            print(f'Num Clusters: {n_clusters}, Silhouette Score: {score:.3f}')
            if score > best_score:
                best_score = score
                best_n_clusters = n_clusters
        except ValueError:
            continue
    
    kmeans_level_2 = KMeans(n_clusters=best_n_clusters, random_state=42)
    sub_cluster_labels = kmeans_level_2.fit_predict(firm_features_normalized[cluster_indices])
    second_level_labels[cluster_indices] = sub_cluster_labels + (cluster * 10)

pca = IncrementalPCA(n_components=2, batch_size=200)
reduced_features = pca.fit_transform(firm_features_normalized)
markers = cycle(['o', 's', 'v', '^', '<', '>', 'P', '*', 'X', 'D'])

plt.figure(figsize=(10, 6))
for cluster in np.unique(second_level_labels):
    marker = next(markers)
    plt.scatter(reduced_features[second_level_labels == cluster, 0],
                reduced_features[second_level_labels == cluster, 1],
                label=f'Sub-Cluster {cluster}',
                marker=marker)

texts = []
for i, (firm, cluster) in enumerate(zip(firm_names, kmeans_labels)):
    texts.append(plt.text(reduced_features[i, 0], reduced_features[i, 1], firm, fontsize=8, alpha=0.75))

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))

#scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=second_level_labels, cmap='tab20')
#plt.legend(*scatter.legend_elements(), title="Sub-Clusters")
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Nested KMeans Clustering Visualization with PCA')
plt.legend()
plt.savefig(f'results/new_kmeans_hierarchical_2_N_clusters')
plt.show()